In [1]:
from Bio import SearchIO,SeqIO
import ete3
import os,functools,json

ncbi=ete3.ncbi_taxonomy.NCBITaxa()

In [2]:
acc_to_gi={} ## accession to gi dict
acc_to_taxid={} ## accession to taxid dict

for line in open('/Users/evogytis/Documents/skeeters/data/acc2gi.txt','r'):
    acc,gi,taxid=line.strip('\n').split('\t')
    acc_to_gi[acc]=gi
    acc_to_taxid[acc]=int(taxid)

In [3]:
# J=json.load(open('/Users/evogytis/Documents/skeeters/treemap/displayed_taxa.json','r')) ## load designated treemap branches
# branches={b['taxid']:b for b in J} ## flat list of branches indexed by taxid
# branches['no_hit']={'taxonomy':'no blast hit','taxid':'no_hit'}

# for b in branches:
#     branches[b]['attrs']={} ## create attributes dict

# local_path='/Users/evogytis/Dropbox/Jupyter_notebooks/Biohub/California_mosquitoes/s3_bucket/'

# sampleCount=1
# for subdir, dirs, files in os.walk(local_path): ## iterate through samples
#     sample=os.path.basename(subdir) ## sample name is folder name
#     if sample.startswith('CMS'):
#         for b in branches:
#             branches[b]['attrs'][sample]={'read_count':0,'contig_count':0} ## create a category for each sample

#         search={}

#         print('\n%d %s'%(sampleCount,sample))
#         for fname in files: ## iterate through files in subfolders
#             fpath=os.path.join(subdir, fname)
#             if 'blast' in fname: ## only interested in blast outputs
#                 if 'gsnap' in fname:
#                     blast_type='nt'
#                 elif 'rapsearch' in fname:
#                     blast_type='aa'

#                 blast_search=SearchIO.parse(fpath,format='blast-tab') ## read blast output
#                 search[blast_type]=SearchIO.to_dict(blast_search) ## turn to dict

#             elif 'fasta' in fname: ## fasta contains contigs
#                 contigs=SeqIO.parse(fpath,format='fasta') ## parse fasta contigs
#                 seqs=SeqIO.to_dict(contigs) ## convert to dict
#                 print('total number of contigs in sample: %s'%(len(seqs)))

#         if len(search)==2 and contigs and 'water' not in sample.lower(): ## once done with sample, if there's nt and nr blast results, contigs are loaded and they're not controls - proceed
#             for searchType in search: ## iterate over nt or nr blast results
#                 no_hits=len(seqs)-len(search[searchType]) ## no hits is (total number of contigs - blast hits)

#                 if searchType=='aa':
#     #                 if 'attrs' not in branches['no_hit']: ## no-hit branch doesn't have attributes yet
#     #                     branches['no_hit']['attrs']={}
#                     branches['no_hit']['attrs'][sample]['contig_count']=no_hits ## assign sample contig_count
#                     print('number of contigs without blastx hit: %d'%(no_hits))

#                 for contig in search[searchType]: ## iterate through contigs blasted
#                     blast_hit={} ## keep a record of hits
#                     bitscores={} ## keep a record of bitscores
#                     percent_id={} ## keep a record of percent identities
#                     for hit in search[searchType][contig]: ## iterate over hits to contig
#                         hit_id=hit.id

#                         ## want to keep hits where best HSP is <1e-2 and longest match is >200 nt
#                         if min([h.evalue for h in hit.hsps])<1e-2 and max([len(f) for f in hit.fragments])>200:
#                             blast_hit[hit_id]=hit
#                             bitscores[hit_id]=min([k.bitscore for k in hit.hsps]) ## keep track of worst HSP bitscore
#                             percent_id[hit_id]=min([k.ident_pct for k in hit.hsps]) ## keep track of worst HSP percent identity

#                     cutoff=0.1 ## within 10% of top hit
#                     filteredHits=filter(lambda k: max(bitscores.values())*(1.0-cutoff)<=bitscores[k], blast_hit) ## filter hits down to those whose bitscore within 10% of the best bitscore
#                     hitLineages=[ncbi.get_lineage(acc_to_taxid[k]) for k in filteredHits] ## get lineage of every hit
#                     hitLineages=list(filter(lambda x: 7157 not in x, hitLineages)) ## filter out lineages with mosquito in its path

#                     if searchType=='aa': ## if nr
#                         hitLineages=list(filter(lambda x: 10239 in x, hitLineages)) ## keep viruses (nr)
#                     elif searchType=='nt': ## if nt
#                         hitLineages=list(filter(lambda x: 10239 not in x, hitLineages)) ## ignore viruses (nt)

#                     if len(list(hitLineages))>0: ## hits exist
#                         ranksPresent=functools.reduce(set.intersection,map(set,hitLineages)) ## reduce lineages seen down to those that are shared by all hits

#                         lineages=[ncbi.get_lineage(lin) for lin in ranksPresent] ## get full lineage of each taxonomic rank that's shared across all hits
#                         mrca_path=[lin for lin in lineages if len(lin)==max(map(len,lineages))][-1] ## longest lineage path is the most recent common ancestor

#                         for rank in mrca_path[::-1]: ## invert ancestor path, so ranks iterated from most recent to oldest
#                             if rank in branches: ## rank is present amongst those designated for treemap
#                                 branch=branches[rank] ## fetch branch from treemap

#     #                             if 'attrs' not in branch:
#     #                                 branch['attrs']={} ## branch doesn't have attrs

#     #                             if sample not in branch['attrs']:
#     #                                 branch['attrs'][sample]={} ## branch hasn't seen sample

#     #                             if 'contig_count' not in branch['attrs'][sample]:
#     #                                 branch['attrs'][sample]['contig_count']=0 ## branch hasn't seen contig_count in sample
#                                 branch['attrs'][sample]['contig_count']+=1 ## add to contig_count
#                                 break ## no need to iterate further
#         sampleCount+=1

In [4]:
J=json.load(open('/Users/evogytis/Documents/skeeters/treemap/displayed_taxa.json','r')) ## load designated treemap branches
branches={b['taxid']:b for b in J} ## flat list of branches indexed by taxid
branches['no_hit']={'taxonomy':'no blast hit','taxid':'no_hit'}

samples=[line.split(',')[1] for line in open('/Users/evogytis/Documents/skeeters/data/TableS1data.csv','r').readlines()[1:]] ## sample names

for b in branches:
    branches[b]['attrs']={sample:{'contig_count':0,'read_count':0} for sample in samples} ## create attributes dict, split by sample
    branches[b]['attrs']['colour']='slategrey'

contig_classification='/Users/evogytis/Documents/skeeters/data/reblast_contigs_no_arthopods.csv'
# contig_classification='/Users/evogytis/Documents/skeeters/data/reblast_contigs.csv'
import csv
for line in csv.reader(open(contig_classification,'r')):
#     l=line.strip('\n').split(',')
    if line[0]=='qseqid':
        header={x:i for i,x in enumerate(line)}
    else:
        
        try:
            taxid=line[header['taxid']]
            sample=line[header['sample']]
            
            if 'CMS002_0Water' in sample:
                sample=sample.replace('CMS002_0Water','CMS002_Water')
            
#             print(ncbi.get_lineage(taxid))
            for step in ncbi.get_lineage(taxid)[::-1]: ## iterate through lineage in reverse (recent to old)
                if step in branches: ## lineage is designated for visualisation
                    branches[step]['attrs'][sample]['contig_count']+=1 ## add to count
                    break ## break for loop
                    
        except ValueError:
            print(line[header['qseqid']],line[header['sample']])
    

In [5]:
report_path='/Users/evogytis/Documents/skeeters/data/mosquito_reports.csv'

reads={}
for line in open(report_path,'r'):
    l=line.strip('\n').split(',')
    if l[0]=='water':
        header={x:i for i,x in enumerate(l)} ## map header label to index
    else:
        if l[header['tax_level']]=='1' and l[header['water']]=='False': ## only species and non-waters
            sample=l[header['sample_name']] ## get sample name
            taxid=int(l[header['tax_id']]) ## get assigned taxid
            
            nr_reads=float(l[header['NR_r']]) ## number of reads in NR
            nt_reads=float(l[header['NT_r']]) ## number of reads in NT
            
#             if sample not in reads: ## sample not seen before
#                 reads[sample]={}
            
#             if taxid not in reads[sample]: ## taxid not seen in sample
#                 reads[sample][taxid]={}
#             else:
#                 print(sample,taxid,'already seen') ## warn in case there's more than one taxid in sample
                
#             reads[sample][taxid]['nr_reads']=nr_reads
#             reads[sample][taxid]['nt_reads']=nt_reads ## assign numbers
            
            mrca_path=ncbi.get_lineage(taxid)
            proceed=True
            if 7157 in mrca_path:
                proceed=False
            
            if 10239 in mrca_path:
                n_reads=nr_reads ## virus, use NR
            elif 10239 not in mrca_path:
                n_reads=nt_reads ## not virus, use NT
            
            for rank in mrca_path[::-1]: ## invert ancestor path, so ranks iterated from most recent to oldest
                if proceed==True and rank in branches: ## rank is present amongst those designated for treemap
                    branch=branches[rank] ## fetch branch from treemap

#                     if 'attrs' not in branch:
#                         branch['attrs']={} ## branch doesn't have attrs

#                     if sample not in branch['attrs']:
#                         branch['attrs'][sample]={} ## branch hasn't seen sample

#                     if 'read_count' not in branch['attrs'][sample]:
#                     branch['attrs'][sample]['read_count']=0 ## branch hasn't seen contig_count in sample 
                    if sample in branch['attrs']:
                        branch['attrs'][sample]['read_count']+=n_reads ## add to contig_count
                        if 2 in mrca_path: ## bacteria
                            branch['attrs']['colour']='#82689C'
#                         '#4BC067' ## plants
                        elif 2759 in mrca_path: ## eukaryotes
                            branch['attrs']['colour']='#4B8AC0'
                        elif 10239 in mrca_path: ## viruses
                            branch['attrs']['colour']='#D07558'
                    break ## no need to iterate further
                
                

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:240: UserWarning: taxid 189363 was translated into 2506096
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:240: UserWarning: taxid 497866 was translated into 2497756
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:240: UserWarning: taxid 2339234 was translated into 2494702
  warnings.warn("taxid %s was translated into %s" %(taxid, merged_conversion[taxid]))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ete3/ncbi_taxonomy/ncbiquery.py:240: UserWarning: taxid 1508232 was translated into 2502029
  warnings.warn("taxid %s was translated into %s" %(taxid, merge

In [6]:
taxa_tree={'children':[branches['no_hit']],'attrs':{'colour':'slategrey'}} ## tree structure (+ add no-hit branch from the beginning)

J=sorted(J,key=lambda k: len(ncbi.get_lineage(k['taxid'])))

for b in branches: ## iterate through flat list of branches
#     if 'attrs' in branches[b]: ## branch has been annotated before
#     print(branches[b]['taxonomy'])
    if branches[b]['taxid']!='no_hit':
        branches[b]['attrs']['read_count']=sum([branches[b]['attrs'][c]['read_count'] for c in branches[b]['attrs'] if 'CMS' in c])
        branches[b]['attrs']['contig_count']=sum([branches[b]['attrs'][c]['contig_count'] for c in branches[b]['attrs'] if 'CMS' in c]) ## compute sum of sample contig counts
#         branches[b]['attrs']['sample_count']=len([c for c in branches[b]['attrs'] if 'CMS' in c])
#     else:
#         branches[b]['attrs']={'contig_count':0} ## nothing landed on the branch
#         branches[b]['attrs']['sample_count']=0
#         branches[b]['attrs']['read_count']=0

for taxon in J: ## iterate over every taxon in treemap
    lineage=ncbi.get_lineage(taxon['taxid']) ## get its lineage
    
    if len(lineage)==1 and taxon not in taxa_tree['children']: ## if root
        taxa_tree['children'].append(taxon) ## add root to tree
        
    for lin in lineage[::-1][1:]: ## iterate through lineage, starting from most recent, ignore first entry (self)
        
        if lin in branches: ## rank present amongst branches
            parent=branches[lin] ## grab parent
            if 'children' not in parent: ## if parent doesn't have children yet - add the attribute
                parent['children']=[]
                
            if taxon not in parent['children']: ## branch wasn't assigned to its parent yet
                parent['children'].append(taxon) ## add child to parent
            
            break

json.dump(taxa_tree,open('/Users/evogytis/Documents/skeeters/treemap/skeeters.json','w'),indent=1) ## write json out to repo


In [7]:
## then run python3 server locally in the folder where treemap.html is located:
## python3 -m http.server 4000
## python -m SimpleHTTPServer 4000

